*Adapted from chapter 3 in the excellent book: Practical Deep Learning*

https://www.practicaldeeplearning.ai/

*Highly recommended*

# Tensor Flow, Computer Vision, Image Classifer, Colab-Notebook

### Required (to make a model):
two zipped folders:
- train 
- val

In each of those folders (train and val) is another pair of (or set of) folders, one for each classification-class (e.g. "apples" and "oranges"). 

Each of those folders contains some portion of the overall set of training images. E.g. for an 80/20 split train/validate: put 80% of pics per classificaiton-class in that folder, and 20% in the other.

For upload to colab, the folders should be zipped, as train.zip, val.zip. This notebook is set up to unzip those.

To try out whether the model works given an input picture:

### This Requires (to test on a picture):
a picture (or one for each classification-class, as you like it.) 




### Required Naming:
The formatting of the naming of the images may need to be standardized:
```
image_class.number.filetype
```
e.g.
```
dog.123.jpg
```



## Colab file contents will look something like this (after the train and val folders are unzipped, and with two test-it-out images):
```
orange.1.jpeg	  sample_data  train.zip  val.zip
apple.1.jpg  train        val
```
Note: The sample data folder is a default of google-colab. 


### Reguired: to use the ai model that you make later:
Make sure you donwload it from the colab and save it. 

# Helper Code: Script to add the classification-class name to a folder full of numbered image files.

Taking 1000 pictures is easy! Manually renaming them all is less fun.

Here is a python script to add the classifican-class-name to your images. 

:tada: !

In [ ]:
"""
Code to rename all files in a directory
with the name of the directory appended 
to the file name
with a period between:

e.g.
folder_name.original_name.suffix

e.g.
whalepics.231.jpg
"""

# import your vanilla libraries (no pip installs needed)
import os 
import glob

# get the folder name (whole current working directory path)
full_path = os.getcwd()

"""
reduce full name to just the name of this directory
split the name on "/" and slice off just the last name [-1]
"""
this_directory = full_path.split("/")[-1]

"""
iterate through a list of folder contents obtained with glob
'*' is a wildcard to get all files, but you can substitute
in a specific suffix for a file type
e.g. glob.glob('*.jpg')
"""
for this_file_name in glob.glob('*.*'):

    # optional print old name
    print("old = ", this_file_name)
    
    # make the new name string	
    new_name = this_directory + "." + this_file_name 
    
    # change old name to new neame
    os.rename(this_file_name, new_name)
    
    # optional print the new name
    print("new = ", new_name, "\n")


### Unzip your two folders/directories:
- train
- val

In [ ]:
# unzip training data folder (of pics)
!unzip train.zip

In [ ]:
# unzip validation data folder (of pics)
!unzip val.zip

In [ ]:
# inspect directory
!ls

In [ ]:
# import libraries and packages
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

 # but wait, there's more!
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

Let's place all the configurations up-front. These can be modified in the future based on the dataset of your choice.

In [ ]:

#################
# Configurations
#################

TRAIN_DATA_DIR = 'train/'
VALIDATION_DATA_DIR = 'val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

## Augmentation

TODO: 

Q: Optimal image rotation?

In [ ]:

###########################################
# set up your preprocessing data generator
###########################################

# set up for training data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)

# set up for validation data
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

## Define the Model

In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

# .fit

In [ ]:
model = model_maker()

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])

model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

## Save the Model

# **You need to manually download this from Colab**



In [ ]:
!ls

# .predict

## Model Prediction (from your saved model)

Now that you have a trained model, you might eventually want to use it later for your application. We can now load this model anytime and classify an image. The Keras function `load_model`, as the name suggests loads the model. 

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Save your model in .h5 format (one file, not a directory)
model.save('model.h5')

# Load the .h5 model file and test it
model = load_model('model.h5')

Now let’s try loading our original sample images and see what results we get.

In [ ]:
!ls

# Test a picture

In [ ]:
# select test image
img_path = 'orange.1.jpeg'

# Preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  

# predict
prediction = model.predict(preprocessed_img)

# print output
print(prediction)
print(validation_generator.class_indices)

# Test an "apple" picture

In [ ]:
# select test image
img_path = 'apple.1.jpg'

# Preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  

# predict
prediction = model.predict(preprocessed_img)

# print output
print(prediction)
print(validation_generator.class_indices)

## Save Tensorflow Model (not yet .tflite)

In [ ]:
export_dir = 'saved_model/'
tf.saved_model.save(model, export_dir)

# inspect
!ls

## Make .ftlite Model (Convert Tensorflow Model to .tflite)

# Make 'model.tflite'

Note: the suffix is fixed, but you can change
the first part of the name,
but the sample code here assumes "model.tflite"


```
YOUR_CHOICE_OF_NAME.tflite
```

In [ ]:
#########################################
# Make Your TFlite Version of your Model
#########################################
# See: https://www.tensorflow.org/lite/convert/

# Save the TF model as directory
TF_model_directory = 'saved_model/'
tf.saved_model.save(model, TF_model_directory)

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model( TF_model_directory ) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# inspect
!ls

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# inspect
!ls

## Now your model.tflite file is ready to be deployed.

# .tflite model 

This code is not the same as the tflite code.

In [ ]:

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# select test image
img_path = 'orange.1.jpg'

# Preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  

# set: input_data = preprocessed image
input_data = preprocessed_img
    
# y: using model, producing y from X
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

# inspection
print("Output is probability of identification.")
print(tflite_results)


Output is probability of identification.
y = [[0.5410198  0.45898017]]
